In [1]:
import numpy as np 
import matplotlib.pyplot as plt
import glob
import cv2

from keras.models import Model, Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
import seaborn as sns
from keras.applications.vgg16 import VGG16
import os

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [2]:
print(os.listdir("C:\\Users\Administrator\Desktop\Cacao V4 model\Cacao V4 dataset/"))

SIZE = 256  #Resize images

#Capture training data and labels into respective lists
train_images = []
train_labels = [] 

for directory_path in glob.glob("C:\\Users\Administrator\Desktop\Cacao V4 model\Cacao V4 dataset/train/*"):
    label = directory_path.split("\\")[-1]
    print(label)
    for img_path in glob.glob(os.path.join(directory_path, "*.jpg")):
        print(img_path)
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)       
        img = cv2.resize(img, (SIZE, SIZE))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        train_images.append(img)
        train_labels.append(label)
        
#Convert lists to arrays        
train_images = np.array(train_images)
train_labels = np.array(train_labels)

['test', 'train', 'val']
black_pod_rot
C:\Users\Administrator\Desktop\Cacao V4 model\Cacao V4 dataset/train\black_pod_rot\black_pod_rot_1.jpg
C:\Users\Administrator\Desktop\Cacao V4 model\Cacao V4 dataset/train\black_pod_rot\black_pod_rot_10.jpg
C:\Users\Administrator\Desktop\Cacao V4 model\Cacao V4 dataset/train\black_pod_rot\black_pod_rot_100.jpg
C:\Users\Administrator\Desktop\Cacao V4 model\Cacao V4 dataset/train\black_pod_rot\black_pod_rot_103.jpg
C:\Users\Administrator\Desktop\Cacao V4 model\Cacao V4 dataset/train\black_pod_rot\black_pod_rot_11.jpg
C:\Users\Administrator\Desktop\Cacao V4 model\Cacao V4 dataset/train\black_pod_rot\black_pod_rot_12.jpg
C:\Users\Administrator\Desktop\Cacao V4 model\Cacao V4 dataset/train\black_pod_rot\black_pod_rot_13.jpg
C:\Users\Administrator\Desktop\Cacao V4 model\Cacao V4 dataset/train\black_pod_rot\black_pod_rot_14.jpg
C:\Users\Administrator\Desktop\Cacao V4 model\Cacao V4 dataset/train\black_pod_rot\black_pod_rot_15.jpg
C:\Users\Administrator\D

C:\Users\Administrator\Desktop\Cacao V4 model\Cacao V4 dataset/train\black_pod_rot\black_pod_rot_98.jpg
C:\Users\Administrator\Desktop\Cacao V4 model\Cacao V4 dataset/train\black_pod_rot\black_pod_rot_99.jpg
healthy
C:\Users\Administrator\Desktop\Cacao V4 model\Cacao V4 dataset/train\healthy\healthy_1.jpg
C:\Users\Administrator\Desktop\Cacao V4 model\Cacao V4 dataset/train\healthy\healthy_10.jpg
C:\Users\Administrator\Desktop\Cacao V4 model\Cacao V4 dataset/train\healthy\healthy_100.jpg
C:\Users\Administrator\Desktop\Cacao V4 model\Cacao V4 dataset/train\healthy\healthy_103.jpg
C:\Users\Administrator\Desktop\Cacao V4 model\Cacao V4 dataset/train\healthy\healthy_11.jpg
C:\Users\Administrator\Desktop\Cacao V4 model\Cacao V4 dataset/train\healthy\healthy_12.jpg
C:\Users\Administrator\Desktop\Cacao V4 model\Cacao V4 dataset/train\healthy\healthy_13.jpg
C:\Users\Administrator\Desktop\Cacao V4 model\Cacao V4 dataset/train\healthy\healthy_14.jpg
C:\Users\Administrator\Desktop\Cacao V4 model\C

C:\Users\Administrator\Desktop\Cacao V4 model\Cacao V4 dataset/train\pod_borer\pod_borer_20.jpg
C:\Users\Administrator\Desktop\Cacao V4 model\Cacao V4 dataset/train\pod_borer\pod_borer_22.jpg
C:\Users\Administrator\Desktop\Cacao V4 model\Cacao V4 dataset/train\pod_borer\pod_borer_23.jpg
C:\Users\Administrator\Desktop\Cacao V4 model\Cacao V4 dataset/train\pod_borer\pod_borer_24.jpg
C:\Users\Administrator\Desktop\Cacao V4 model\Cacao V4 dataset/train\pod_borer\pod_borer_25.jpg
C:\Users\Administrator\Desktop\Cacao V4 model\Cacao V4 dataset/train\pod_borer\pod_borer_26.jpg
C:\Users\Administrator\Desktop\Cacao V4 model\Cacao V4 dataset/train\pod_borer\pod_borer_27.jpg
C:\Users\Administrator\Desktop\Cacao V4 model\Cacao V4 dataset/train\pod_borer\pod_borer_28.jpg
C:\Users\Administrator\Desktop\Cacao V4 model\Cacao V4 dataset/train\pod_borer\pod_borer_29.jpg
C:\Users\Administrator\Desktop\Cacao V4 model\Cacao V4 dataset/train\pod_borer\pod_borer_3.jpg
C:\Users\Administrator\Desktop\Cacao V4 m

In [3]:
# Capture test/validation data and labels into respective lists

test_images = []
test_labels = [] 
for directory_path in glob.glob("images/validation/*"):
    fruit_label = directory_path.split("\\")[-1]
    for img_path in glob.glob(os.path.join(directory_path, "*.jpg")):
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        img = cv2.resize(img, (SIZE, SIZE))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        test_images.append(img)
        test_labels.append(fruit_label)

#Convert lists to arrays                
test_images = np.array(test_images)
test_labels = np.array(test_labels)

In [8]:
#Encode labels from text to integers.
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(test_labels)
test_labels_encoded = le.transform(test_labels)
le.fit(train_labels)
train_labels_encoded = le.transform(train_labels)

In [13]:
#Split data into test and train datasets (already split but assigning to meaningful convention)
x_train, y_train, x_test, y_test = train_images, train_labels_encoded, test_images, test_labels_encoded


In [14]:
# Normalize pixel values to between 0 and 1
x_train, x_test = x_train / 256.0, x_test / 256.0

In [15]:
#One hot encode y values for neural network. 
from keras.utils import to_categorical
y_train_one_hot = to_categorical(y_train)
y_test_one_hot = to_categorical(y_test)

ValueError: zero-size array to reduction operation maximum which has no identity

In [ ]:
VGG_model = VGG16(weights='imagenet', include_top=False, input_shape=(SIZE, SIZE, 3))


In [ ]:
for layer in VGG_model.layers:
	layer.trainable = False
    
VGG_model.summary()  #Trainable parameters will be 0


In [ ]:
#Now, let us use features from convolutional network for RF
feature_extractor=VGG_model.predict(x_train)

In [ ]:
features = feature_extractor.reshape(feature_extractor.shape[0], -1)

In [ ]:
X_for_RF = features #This is our X input to RF


In [ ]:
#RANDOM FOREST
from sklearn.ensemble import RandomForestClassifier
RF_model = RandomForestClassifier(n_estimators = 50, random_state = 42)


In [ ]:
# Train the model on training data
RF_model.fit(X_for_RF, y_train) #For sklearn no one hot encoding

In [ ]:
#Send test data through same feature extractor process
X_test_feature = VGG_model.predict(x_test)
X_test_features = X_test_feature.reshape(X_test_feature.shape[0], -1)

In [ ]:
prediction_RF = RF_model.predict(X_test_features)
#Inverse le transform to get original label back. 
prediction_RF = le.inverse_transform(prediction_RF)